In [1]:
import os
os.environ["CUDA_DEVICE_OREDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
# os.environ["TF_CPP_MIN_LOG_LEVEL"] = '3'
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as dataset
# from torchsummary import summary
# from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import random
print(torch.__version__, torchvision.__version__)
print(torch.cuda.get_device_name(0))

1.8.0 0.2.2
TITAN RTX


In [2]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

max_epochs = 30
print(device)

cuda:0


In [3]:
# randomness 제어 
# https://hoya012.github.io/blog/reproducible_pytorch/
seed = 0
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(seed)
random.seed(seed)

In [4]:
# dataset load

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
])
dataset_path = "/media/2/Network/Imagenet_dup/"
# imagenet data load
train_dataset = dataset.ImageFolder(root=dataset_path+"train",
                                       transform=transform)
val_dataset = dataset.ImageFolder(root=dataset_path+"val",
                                       transform=transform)
train_dataloader = torch.utils.data.DataLoader(train_dataset,
                                        batch_size=16,
                                        shuffle=False,
                                        num_workers=0)
val_dataloader = torch.utils.data.DataLoader(val_dataset,
                                        batch_size=16,
                                        shuffle=False,
                                        num_workers=0)
print(len(train_dataloader),len(val_dataloader))

80073 3125


In [6]:
# model 로드 
vgg16_bn = torchvision.models.vgg16_bn(pretrained=True)
vgg16_bn.to(device)

Downloading: "https://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to /home/hwbae0326/.cache/torch/hub/checkpoints/vgg16_bn-6c64b313.pth
100.0%


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

In [7]:
# # only conv5_1 is trainable
# ct = 0
# layer_level = 1
# back_layer = []
# for layer in vgg16_bn.children():
#     print(ct,layer)
#     if layer_level is 1:
#         for layer_1 in layer:    
#             print("layer 1",ct,layer_1)
#             for param in layer_1.parameters():
#                 if ct != 34:
#                     param.requires_grad = False
#                     print(ct,"layer false")
#                 else :
#                     param.requires_grad = True
#                     print(ct,"layer true")
#             ct +=1
#     elif layer_level is 2:
#         print("layer 2",layer)  
#         for param in layer_1.parameters():
#             param.requires_grad = False
#             print(ct,"layer false")
#         ct +=1
#     elif layer_level is 3:
#         for layer_1 in layer: 
#             print("layer 3",layer_1) 
#             for param in layer_1.parameters():
#                 param.requires_grad = False
#                 print(ct,"layer false")
#             ct+=1
#     layer_level +=1
    

In [8]:
# summary(vgg16_bn.to(device),input_size=(3,224,224))

In [9]:
criterion = torch.nn.CrossEntropyLoss(size_average = None).to(device)
optimizer = torch.optim.SGD(vgg16_bn.parameters(),lr=0.005)
lr_sche = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.9)


In [10]:
# evaluation pahse
val_losses=[]
vgg16_bn.eval()
total = 0
correct =0
with torch.no_grad():
    for i, data in enumerate(val_dataloader):
        inputs,labels = data
        inputs,labels = inputs.to(device), labels.to(device)
        
        y_hat = vgg16_bn(inputs)
        _, predicted = torch.max(y_hat.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        val_loss = criterion(y_hat, labels)
        val_losses.append(val_loss.item())
        if(i%100 == 99):
            print(".",end="")
    print("")
print("acc of vgg16 on imagenet : %.4f %%" %(100*correct/total))    

...............................
acc of vgg16 on imagenet : 73.3600 %
